In [8]:
#pip install pykeen

  DEPRECATION: click-default-group is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: C:\Users\bless\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



  Using cached pykeen-1.10.1-py3-none-any.whl (739 kB)
  Using cached dataclasses_json-0.5.13-py3-none-any.whl (26 kB)
     ---------------------------------------- 97.9/97.9 kB 2.8 MB/s eta 0:00:00
  Using cached click-default-group-1.2.2.tar.gz (3.3 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 77.1/77.1 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB 3.3 MB/s eta 0:00:00
  Using cached optuna-3.2.0-py3-none-any.whl (390 kB)
  Using cached more_click-0.1.2-py3-none-any.whl (6.7 kB)
  Using cached more_itertools-10.0.0-py3-none-any.whl (55 kB)
  Using cached pystow-0.5.0-py3-none-any.whl (37 kB)
  Using cached docdata-0.0.3-py3-none-any.whl (5.8 kB)
  Using cached class_resolver-0.4.2-py3-none-any.whl (24 kB)
     -------------------------------------- 144.7/144.7 kB 4.2 MB/s eta 0:00:00
  Using cached rexmex-0.1.3.tar.gz (21 kB)
  Installing bu

In [9]:
#pip install neo4j


  Using cached neo4j-5.10.0.tar.gz (187 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.10.0-py3-none-any.whl size=259319 sha256=b740b0fe46126aee64cd0c704d4282187a28a137d7612113701b4297ce6097a5
  Stored in directory: c:\users\bless\appdata\local\pip\cache\wheels\15\e7\63\91fd6c4e68d0a4cd8d3aaf55f181f978488158ca3bdd5bbcb7
Successfully built neo4j
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: C:\Users\bless\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [21]:
# from pykeen.pipeline import pipeline

# result = pipeline(
#     model='TransE',
#     dataset='nations',
# )

In [22]:
# Define Neo4j connections
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = '12345678'
database = 'hetionet1'
driver = GraphDatabase.driver(host,auth=(user, password), database = database)
                                         

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [23]:
data = run_query("""
match (s)-[r]->(t) return toString(id(s)) as source, toString(id(t)) as target, type(r) as type
""")

In [24]:
data.head()

,source,target,type
0,0,12590,interacts
1,0,8752,interacts
2,0,7915,interacts
3,0,21711,interacts
4,0,6447,interacts


In [17]:
#pip install --upgrade pykeen


Note: you may need to restart the kernel to use updated packages.


In [25]:
from pykeen.triples import TriplesFactory


tf = TriplesFactory.from_labeled_triples(
  data[["source", "type", "target"]].values,
  create_inverse_triples=False,
  entity_to_id=None,
  relation_to_id=None,
  compact_id=False,
  filter_out_candidate_inverse_relations=True,
  metadata=None,
)


In [26]:
tf

TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=561721)

In [27]:
training, testing, validation = tf.split([.7, .2, .1])

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [374117, 112344, 56173]


In [28]:
training

TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204)

In [29]:
testing

TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=112344)

In [30]:
validation

TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=56173)

# Running QuatE and ProjE model on the Hetionet data

In [31]:
from pykeen.pipeline import pipeline
import torch

# Assuming you already have the necessary imports and data loaded

# Set the device to 'cuda' if a GPU is available, otherwise use 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create the pipeline with the desired model and other settings
result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='QuatE',
    stopper='early',
    epochs=3,
    dimensions=512,
    random_seed=420,
    device=device,  # Set the device to use the GPU
)

# Print the result or perform other operations with the model and result
print(result)


# Create the pipeline with the desired model and other settings
result1 = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='ProjE',
    stopper='early',
    epochs=3,
    dimensions=512,
    random_seed=420,
    device=device,  # Set the device to use the GPU
)

# Print the result or perform other operations with the model and result
print(result)


INFO:pykeen.pipeline.api:Using device: cpu
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\bless\.data\pykeen\checkpoints\best-model-weights-1fb69115-4b07-494f-8797-d97bbe1bd261.pt


Training epochs on cpu:   0%|          | 0/3 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/1536 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/1536 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/1536 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/1536 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.


Evaluating on cpu:   0%|          | 0.00/112k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 387.98s seconds


PipelineResult(random_seed=420, model=QuatE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): QuatEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (regularizer): LpRegularizer()
      (_embeddings): Embedding(19930, 400)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (regularizer): LpRegularizer()
      (_embeddings): Embedding(10, 400)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x000001B7A8E90550>, losses=[0.6240022398026971, 0.27454931514997344, 0.18387955038148598], metric_results=<pykeen.evaluation.rank_based_evaluator.RankBasedMetricResults object at 0x000001B7AFE92950>, train_seconds=379.334023475647, evaluate_seconds=9005.172942638397, stopper=EarlyStopper(evaluator=<pykeen.evaluation.rank_based_evalua

In [156]:
result.save_to_directory('QuatE_results')
result1.save_to_directory('ProjE_results')
print(result1)

INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204) to file:///C:/Users/bless/OneDrive/MS_Studies/AML/NEO4J/CODE/QuatE_results/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/Users/bless/OneDrive/MS_Studies/AML/NEO4J/CODE/QuatE_results
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204) to file:///C:/Users/bless/OneDrive/MS_Studies/AML/NEO4J/CODE/ProjE_results/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/Users/bless/OneDrive/MS_Studies/AML/NEO4J/CODE/ProjE_results


PipelineResult(random_seed=420, model=ProjE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): ProjEInteraction(
    (inner_non_linearity): Tanh()
  )
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(19930, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(10, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x000001B7A8E923B0>, losses=[0.7512686726792405, 0.5438912059568489, 0.42557954918205115], metric_results=<pykeen.evaluation.rank_based_evaluator.RankBasedMetricResults object at 0x000001B7AFE5CCD0>, train_seconds=50.35933756828308, evaluate_seconds=387.988242149353, stopper=EarlyStopper(evaluator=<pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator object at 0x00000

In [39]:
# !pip install torch torchvision


In [1]:
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)  # Check if it outputs 'cuda' if GPU is available, or 'cpu' if not.

# # Additional code using the device (e.g., moving tensors to GPU)
# # Importing Pytorch
# # To print Cuda version
# print("Pytorch CUDA Version is ", torch.version.cuda)

cpu
Pytorch CUDA Version is  None


In [46]:
# # setting device on GPU if available, else CPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('Using device:', device)
# print()

# #Additional Info when using cuda
# if device.type == 'cuda':
#     print(torch.cuda.get_device_name(0))
#     print('Memory Usage:')
#     print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
#     print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



# Usecase1: Prediction of Upregulated genes in Breast Cancer

In [248]:
#from pykeen import predict
from pykeen.predict import predict_target
compound_id = run_query("""
MATCH (s:Disease)
WHERE s.name = "breast cancer"
RETURN toString(id(s)) as id
""")['id'][0]

compound_id

#df
#df['in_training']

'3099'

### Analysis using QuatE

In [249]:
df = predict_target(model=result.model,
                    head=compound_id,
                    relation='upregulates',
                    triples_factory=result.training,)
print(df)
#what is df

TargetPredictions(df=       tail_id     score tail_label
5398      5398  3.324914      15544
15816    15816  3.171596       5812
918        918  3.019877      10948
1146      1146  3.018856      11183
5228      5228  2.867625      15371
...        ...       ...        ...
10822    10822 -2.560827      21080
968        968 -2.589779        110
11547    11547 -2.703009      21814
13167    13167 -2.710057       3099
12987    12987 -2.876242       2912

[19930 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='tail', other_columns_fixed_ids=(13167, 9))


### Analysis using ProjE

In [250]:
df1 = predict_target(model=result1.model,
                    head=compound_id,
                    relation='upregulates',
                    triples_factory=result1.training,)
print(df1)

TargetPredictions(df=       tail_id     score tail_label
9817      9817  4.429758      20057
2056      2056  4.248618        121
7566      7566  4.181810      17758
3266      3266  4.173839       1336
5504      5504  4.171958      15651
...        ...       ...        ...
19316    19316 -3.103631       9373
8919      8919 -3.104685      19147
16139    16139 -3.108089       6148
1763      1763 -3.174391      11811
8948      8948 -3.192385      19177

[19930 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='tail', other_columns_fixed_ids=(13167, 9))


In [251]:
compound_id

'3099'

In [252]:
pred_filtered = df.filter_triples(training)


In [254]:
pred_filtered

TargetPredictions(df=       tail_id     score tail_label
15816    15816  3.171596       5812
11515    11515  2.790679      21782
12568    12568  2.772505       2480
7771      7771  2.731557      17966
8041      8041  2.725287      18240
...        ...       ...        ...
10822    10822 -2.560827      21080
968        968 -2.589779        110
11547    11547 -2.703009      21814
13167    13167 -2.710057       3099
12987    12987 -2.876242       2912

[19756 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='tail', other_columns_fixed_ids=(13167, 9))

In [255]:
pred_filtered1 = df1.filter_triples(training)
pred_filtered1

TargetPredictions(df=       tail_id     score tail_label
9817      9817  4.429758      20057
2056      2056  4.248618        121
7566      7566  4.181810      17758
5504      5504  4.171958      15651
11332    11332  4.161258      21599
...        ...       ...        ...
19316    19316 -3.103631       9373
8919      8919 -3.104685      19147
16139    16139 -3.108089       6148
1763      1763 -3.174391      11811
8948      8948 -3.192385      19177

[19756 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='tail', other_columns_fixed_ids=(13167, 9))

In [257]:
pred_annotated = pred_filtered.add_membership_columns(validation=validation, testing=testing, training=training)

In [258]:
pred_annotated1 = pred_filtered1.add_membership_columns(validation=validation, testing=testing, training=training)

In [259]:
pred_annotated

TargetPredictions(df=       tail_id     score tail_label  in_validation  in_testing  in_training
15816    15816  3.171596       5812          False       False        False
11515    11515  2.790679      21782          False       False        False
12568    12568  2.772505       2480           True       False        False
7771      7771  2.731557      17966          False       False        False
8041      8041  2.725287      18240          False       False        False
...        ...       ...        ...            ...         ...          ...
10822    10822 -2.560827      21080          False       False        False
968        968 -2.589779        110          False       False        False
11547    11547 -2.703009      21814          False       False        False
13167    13167 -2.710057       3099          False       False        False
12987    12987 -2.876242       2912          False       False        False

[19756 rows x 6 columns], factory=TriplesFactory(num_entities=1993

In [260]:
pred_annotated1

TargetPredictions(df=       tail_id     score tail_label  in_validation  in_testing  in_training
9817      9817  4.429758      20057          False       False        False
2056      2056  4.248618        121          False       False        False
7566      7566  4.181810      17758          False       False        False
5504      5504  4.171958      15651          False       False        False
11332    11332  4.161258      21599          False       False        False
...        ...       ...        ...            ...         ...          ...
19316    19316 -3.103631       9373          False       False        False
8919      8919 -3.104685      19147          False       False        False
16139    16139 -3.108089       6148          False       False        False
1763      1763 -3.174391      11811          False       False        False
8948      8948 -3.192385      19177          False       False        False

[19756 rows x 6 columns], factory=TriplesFactory(num_entities=1993

In [261]:
df.filter_triples(testing)


TargetPredictions(df=       tail_id     score tail_label
5398      5398  3.324914      15544
15816    15816  3.171596       5812
918        918  3.019877      10948
1146      1146  3.018856      11183
5228      5228  2.867625      15371
...        ...       ...        ...
10822    10822 -2.560827      21080
968        968 -2.589779        110
11547    11547 -2.703009      21814
13167    13167 -2.710057       3099
12987    12987 -2.876242       2912

[19883 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='tail', other_columns_fixed_ids=(13167, 9))

In [262]:
df1.filter_triples(testing)

TargetPredictions(df=       tail_id     score tail_label
9817      9817  4.429758      20057
2056      2056  4.248618        121
7566      7566  4.181810      17758
3266      3266  4.173839       1336
5504      5504  4.171958      15651
...        ...       ...        ...
19316    19316 -3.103631       9373
8919      8919 -3.104685      19147
16139    16139 -3.108089       6148
1763      1763 -3.174391      11811
8948      8948 -3.192385      19177

[19883 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='tail', other_columns_fixed_ids=(13167, 9))

# Prediction results for QuatE

In [42]:
# in_training = [tail_id in result.training.entity_to_id for tail_id in df['tail_id']]

# # Add the 'in_training' column to the DataFrame
# df['in_training'] = in_training

# # Rename the columns to match your desired output
# df = df.rename(columns={'tail_id': 'tail_label', 'score': 'tail_predict'})

# # Print the DataFrame
# print(df.head(5))

      tail_label  tail_predict tail_label  in_training
3755        3755     -3.131956      CREB1        False
5059        5059     -3.163479       E2F1        False
13047      13047     -3.167017      POLD1        False
15879      15879     -3.170648      SMAD3        False
9906        9906     -3.172402     MAPK14        False


In [265]:
df=pred_annotated.df
df.head(30)

,tail_id,score,tail_label,in_validation,in_testing,in_training
15816,15816,3.171596,5812,False,False,False
11515,11515,2.790679,21782,False,False,False
12568,12568,2.772505,2480,True,False,False
7771,7771,2.731557,17966,False,False,False
8041,8041,2.725287,18240,False,False,False
14831,14831,2.670315,4803,False,False,False
18684,18684,2.670084,8737,False,False,False
14855,14855,2.666677,4827,False,True,False
18837,18837,2.663634,8889,False,False,False
7566,7566,2.660922,17758,False,False,False


# Prediction results for ProjE

In [266]:
df1=pred_annotated1.df
df1.head(30)

,tail_id,score,tail_label,in_validation,in_testing,in_training
9817,9817,4.429758,20057,False,False,False
2056,2056,4.248618,121,False,False,False
7566,7566,4.181810,17758,False,False,False
5504,5504,4.171958,15651,False,False,False
11332,11332,4.161258,21599,False,False,False
11450,11450,4.144822,21714,False,False,False
13800,13800,4.143008,3747,False,False,False
11737,11737,4.105857,2200,False,False,False
17042,17042,4.030718,7072,False,False,False
11541,11541,4.020617,21807,False,False,False


### Writing back the results into Neo4j and exporting the results as csv

In [192]:
candidate_nodes = df[df['in_training'] == False].head(50)['tail_label'].to_list()

run_query("""
MATCH (n)
WHERE id(n) = toInteger($compound_id)
UNWIND $candidates as ca
MATCH (c)
WHERE id(c) = toInteger(ca)
MERGE (n)-[:PREDICTED_UPREGULATED_QuatE_breast]->(c)
""", {'compound_id':compound_id, 'candidates': candidate_nodes})

# print(candidate_nodes)
# print(compound_id)

""


In [193]:
candidate_nodes1 = df1[df1['in_training'] == False].head(50)['tail_label'].to_list()

run_query("""
MATCH (n)
WHERE id(n) = toInteger($compound_id)
UNWIND $candidates as ca
MATCH (c)
WHERE id(c) = toInteger(ca)
MERGE (n)-[:PREDICTED_UPREGULATED_ProjE_breast]->(c)
""", {'compound_id':compound_id, 'candidates': candidate_nodes1})

# print(candidate_nodes)
# print(compound_id)

""


In [294]:
run_query("""
MATCH (c:Disease)-[:PREDICTED_UPREGULATED_QuatE_breast]->(d:Gene)
RETURN c.name as disease, d.name as gene
""")

,disease,gene
0,breast cancer,APOBEC3B
1,breast cancer,COG2
2,breast cancer,EXOSC9
3,breast cancer,SQSTM1
4,breast cancer,APP
5,breast cancer,SUV39H1
6,breast cancer,CMTM6
7,breast cancer,RRP1B
8,breast cancer,NMI
9,breast cancer,KLHL21


In [295]:
gene_quat=run_query("""
MATCH (c:Disease)-[:PREDICTED_UPREGULATED_QuatE_breast]->(d:Gene)
RETURN c.name as disease, d.name as gene
""")

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(gene_quat)

# Specify the file name for the CSV export
csv_file = "PREDICTED_UPREGULATED_QuatE_breast.csv"

# Export the DataFrame to CSV
df.to_csv(csv_file, index=False)

print(f"Data has been exported to {csv_file}.")

Data has been exported to PREDICTED_UPREGULATED_QuatE_breast.csv.


In [290]:
run_query("""
MATCH (c:Disease {name: "breast cancer"}),(d:Gene{name:"SMC4"})
WITH c,d
MATCH p=AllShortestPaths((c)-[r:binds|regulates|interacts|upregulates|downregulates|associates*1..4]-(d))
RETURN [n in nodes(p) | n.name] LIMIT 25
""")

,[n in nodes(p) | n.name]
0,"[breast cancer, SMC4]"


In [291]:
run_query("""
MATCH (c:Disease {name: "breast cancer"}),(d:Gene{name:"APOBEC3B"})
WITH c,d
MATCH p=AllShortestPaths((c)-[r:binds|regulates|interacts|upregulates|downregulates|associates*1..4]-(d))
RETURN [n in nodes(p) | n.name] LIMIT 25
""")

,[n in nodes(p) | n.name]
0,"[breast cancer, APOBEC3B]"


In [292]:
run_query("""
MATCH (c:Disease {name: "breast cancer"}),(d:Gene{name:"SQSTM1"})
WITH c,d
MATCH p=AllShortestPaths((c)-[r:binds|regulates|interacts|upregulates|downregulates|associates*1..4]-(d))
RETURN [n in nodes(p) | n.name] LIMIT 25
""")

,[n in nodes(p) | n.name]
0,"[breast cancer, SQSTM1]"


In [269]:
run_query("""
MATCH (c:Disease)-[:PREDICTED_UPREGULATED_ProjE_breast]->(d:Gene)
RETURN c.name as disease, d.name as gene
""")

,disease,gene
0,breast cancer,SPP1
1,breast cancer,WIF1
2,breast cancer,TUBA1A
3,breast cancer,OXA1L
4,breast cancer,LBR
5,breast cancer,CDK4
6,breast cancer,SSBP2
7,breast cancer,JUN
8,breast cancer,TMEM109
9,breast cancer,AGR2


In [297]:
gene_proje=run_query("""
MATCH (c:Disease)-[:PREDICTED_UPREGULATED_ProjE_breast]->(d:Gene)
RETURN c.name as disease, d.name as gene
""")
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(gene_proje)

# Specify the file name for the CSV export
csv_file = "PREDICTED_UPREGULATED_ProjE_breast.csv"

# Export the DataFrame to CSV
df.to_csv(csv_file, index=False)

print(f"Data has been exported to {csv_file}.")

Data has been exported to PREDICTED_UPREGULATED_ProjE_breast.csv.


In [287]:
run_query("""
MATCH (c:Disease {name: "breast cancer"}),(d:Gene{name:"SPP1"})
WITH c,d
MATCH p=AllShortestPaths((c)-[r:binds|regulates|interacts|upregulates|downregulates|associates*1..4]-(d))
RETURN [n in nodes(p) | n.name] LIMIT 25
""")

,[n in nodes(p) | n.name]
0,"[breast cancer, SPP1]"


In [288]:
run_query("""
MATCH (c:Disease {name: "breast cancer"}),(d:Gene{name:"SMC4"})
WITH c,d
MATCH p=AllShortestPaths((c)-[r:binds|regulates|interacts|upregulates|downregulates|associates*1..4]-(d))
RETURN [n in nodes(p) | n.name] LIMIT 25
""")

,[n in nodes(p) | n.name]
0,"[breast cancer, SMC4]"


In [289]:
run_query("""
MATCH (c:Disease {name: "breast cancer"}),(d:Gene{name:"TUBA1A"})
WITH c,d
MATCH p=AllShortestPaths((c)-[r:binds|regulates|interacts|upregulates|downregulates|associates*1..4]-(d))
RETURN [n in nodes(p) | n.name] LIMIT 25
""")

,[n in nodes(p) | n.name]
0,"[breast cancer, TGFBR2, TUBA1A]"
1,"[breast cancer, F3, TUBA1A]"
2,"[breast cancer, GPX4, TUBA1A]"
3,"[breast cancer, NFKBIA, TUBA1A]"
4,"[breast cancer, MAPK13, TUBA1A]"
5,"[breast cancer, SPRY1, TUBA1A]"
6,"[breast cancer, TP53, TUBA1A]"
7,"[breast cancer, CDKN1A, TUBA1A]"
8,"[breast cancer, COMT, TUBA1A]"
9,"[breast cancer, EGFR, TUBA1A]"


# Usecase2: Analysis for Drug connected to gene SMC4

In [271]:
#from pykeen import predict
from pykeen.predict import predict_target
compound_id_g = run_query("""
MATCH (s:Gene)
WHERE s.name = "SMC4"
RETURN toString(id(s)) as id
""")['id'][0]

compound_id_g

#df
#df['in_training']

'345'

In [272]:
df_g = predict_target(model=result.model,
                    relation='downregulates',
                    tail=compound_id_g,
                    triples_factory=result.training,)
print(df_g)
#what is df

TargetPredictions(df=       head_id     score head_label
11547    11547  4.428504      21814
4377      4377  4.383505      14486
7382      7382  4.317099      17565
14396    14396  4.285762       4360
10375    10375  4.283663      20624
...        ...       ...        ...
9789      9789 -1.641454      20029
18837    18837 -1.670421       8889
18903    18903 -1.720475       8954
17148    17148 -1.793026       7182
11449    11449 -2.005073      21711

[19930 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='head', other_columns_fixed_ids=(4, 13507))


In [273]:
df_g_1 = predict_target(model=result1.model,
                    relation='downregulates',
                    tail=compound_id_g,
                    triples_factory=result1.training,)
print(df_g_1)
#what is df

TargetPredictions(df=       head_id     score head_label
13507    13507  6.184639        345
10290    10290  5.133840      20533
11332    11332  5.056638      21599
15138    15138  4.973536       5112
5174      5174  4.964867      15314
...        ...       ...        ...
16364    16364 -1.194883       6372
19624    19624 -1.199694       9689
10075    10075 -1.213256       2032
19621    19621 -1.251976       9685
2133      2133 -1.553366      12177

[19930 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='head', other_columns_fixed_ids=(4, 13507))


In [274]:
pred_filtered_g = df_g.filter_triples(training)
pred_filtered_g_1 = df_g_1.filter_triples(training)
pred_annotated_g = pred_filtered_g.add_membership_columns(validation=validation, testing=testing, training=training)
pred_annotated_g_1 = pred_filtered_g_1.add_membership_columns(validation=validation, testing=testing, training=training)

In [275]:
pred_filtered_g

TargetPredictions(df=       head_id     score head_label
4377      4377  4.383505      14486
14396    14396  4.285762       4360
5377      5377  4.238102      15522
8903      8903  4.218297      19130
11210    11210  4.130627      21472
...        ...       ...        ...
9789      9789 -1.641454      20029
18837    18837 -1.670421       8889
18903    18903 -1.720475       8954
17148    17148 -1.793026       7182
11449    11449 -2.005073      21711

[19901 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='head', other_columns_fixed_ids=(4, 13507))

In [276]:
pred_filtered_g_1

TargetPredictions(df=       head_id     score head_label
13507    13507  6.184639        345
10290    10290  5.133840      20533
11332    11332  5.056638      21599
15138    15138  4.973536       5112
5174      5174  4.964867      15314
...        ...       ...        ...
16364    16364 -1.194883       6372
19624    19624 -1.199694       9689
10075    10075 -1.213256       2032
19621    19621 -1.251976       9685
2133      2133 -1.553366      12177

[19901 rows x 3 columns], factory=TriplesFactory(num_entities=19930, num_relations=10, create_inverse_triples=False, num_triples=393204), target='head', other_columns_fixed_ids=(4, 13507))

In [277]:
pred_annotated_g

TargetPredictions(df=       head_id     score head_label  in_validation  in_testing  in_training
4377      4377  4.383505      14486          False       False        False
14396    14396  4.285762       4360          False       False        False
5377      5377  4.238102      15522          False       False        False
8903      8903  4.218297      19130          False       False        False
11210    11210  4.130627      21472          False       False        False
...        ...       ...        ...            ...         ...          ...
9789      9789 -1.641454      20029          False       False        False
18837    18837 -1.670421       8889          False       False        False
18903    18903 -1.720475       8954          False       False        False
17148    17148 -1.793026       7182          False       False        False
11449    11449 -2.005073      21711          False       False        False

[19901 rows x 6 columns], factory=TriplesFactory(num_entities=1993

In [278]:
pred_annotated_g_1

TargetPredictions(df=       head_id     score head_label  in_validation  in_testing  in_training
13507    13507  6.184639        345          False       False        False
10290    10290  5.133840      20533          False       False        False
11332    11332  5.056638      21599          False       False        False
15138    15138  4.973536       5112          False       False        False
5174      5174  4.964867      15314          False       False        False
...        ...       ...        ...            ...         ...          ...
16364    16364 -1.194883       6372          False       False        False
19624    19624 -1.199694       9689          False       False        False
10075    10075 -1.213256       2032          False       False        False
19621    19621 -1.251976       9685          False       False        False
2133      2133 -1.553366      12177          False       False        False

[19901 rows x 6 columns], factory=TriplesFactory(num_entities=1993

In [279]:
df_g=pred_annotated_g.df
df_g.head(30)

,head_id,score,head_label,in_validation,in_testing,in_training
4377,4377,4.383505,14486,False,False,False
14396,14396,4.285762,4360,False,False,False
5377,5377,4.238102,15522,False,False,False
8903,8903,4.218297,19130,False,False,False
11210,11210,4.130627,21472,False,False,False
11483,11483,4.119243,21750,False,False,False
18135,18135,4.115297,818,False,True,False
2705,2705,4.113011,1278,False,False,False
19247,19247,4.089200,9297,False,False,False
10206,10206,4.073489,20454,False,False,False


In [280]:
df_g_1=pred_annotated_g_1.df
df_g_1.head(30)

,head_id,score,head_label,in_validation,in_testing,in_training
13507,13507,6.184639,345,False,False,False
10290,10290,5.133840,20533,False,False,False
11332,11332,5.056638,21599,False,False,False
15138,15138,4.973536,5112,False,False,False
5174,5174,4.964867,15314,False,False,False
13785,13785,4.919665,3731,False,False,False
15009,15009,4.902075,4985,False,False,False
19180,19180,4.870707,9231,False,False,False
3266,3266,4.836885,1336,False,False,False
19586,19586,4.833000,9646,False,False,False


### Wrting back the results into Neo4j and exporting the results as csv

In [211]:
candidate_nodes_g = df_g[df_g['in_training'] == False].head(50)['head_label'].to_list()

run_query("""
MATCH (n)
WHERE id(n) = toInteger($compound_id)
UNWIND $candidates as ca
MATCH (c)
WHERE id(c) = toInteger(ca) AND "Compound" IN LABELS(c)
MERGE (c)-[:PREDICTED_DOWNREGULATED_QuatE_breast]->(n)
""", {'compound_id':compound_id_g, 'candidates': candidate_nodes_g})

# print(candidate_nodes)
# print(compound_id)

""


In [212]:
candidate_nodes_g_1 = df_g_1[df_g_1['in_training'] == False].head(50)['head_label'].to_list()

run_query("""
MATCH (n)
WHERE id(n) = toInteger($compound_id)
UNWIND $candidates as ca
MATCH (c)
WHERE id(c) = toInteger(ca) AND "Compound" IN LABELS(c)
MERGE (c)-[:PREDICTED_DOWNREGULATED_ProjE_breast]->(n)
""", {'compound_id':compound_id_g, 'candidates': candidate_nodes_g_1})

# print(candidate_nodes)
# print(compound_id)

""


In [281]:
run_query("""
match (n:Compound)-[r:PREDICTED_DOWNREGULATED_QuatE_breast]->(m:Gene) return n.name,type(r),m.name
""")


,n.name,type(r),m.name
0,Ponatinib,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
1,Simvastatin,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
2,Bortezomib,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
3,Dactinomycin,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
4,Temsirolimus,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
5,Digitoxin,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
6,Afatinib,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
7,Sirolimus,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
8,Tretinoin,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4
9,Sorafenib,PREDICTED_DOWNREGULATED_QuatE_breast,SMC4


In [298]:
drug_quate=run_query("""
match (n:Compound)-[r:PREDICTED_DOWNREGULATED_QuatE_breast]->(m:Gene) return n.name,type(r),m.name
""")
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(drug_quate)

# Specify the file name for the CSV export
csv_file = "PREDICTED_DOWNREGULATED_QuatE_breast.csv"

# Export the DataFrame to CSV
df.to_csv(csv_file, index=False)

print(f"Data has been exported to {csv_file}.")

Data has been exported to PREDICTED_DOWNREGULATED_QuatE_breast.csv.


In [282]:
run_query("""
match (n:Compound)-[r:PREDICTED_DOWNREGULATED_ProjE_breast]->(m:Gene) return n.name,type(r),m.name
""")

,n.name,type(r),m.name
0,Perhexiline,PREDICTED_DOWNREGULATED_ProjE_breast,SMC4


In [299]:
drug_proje=run_query("""
match (n:Compound)-[r:PREDICTED_DOWNREGULATED_ProjE_breast]->(m:Gene) return n.name,type(r),m.name
""")
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(drug_proje)

# Specify the file name for the CSV export
csv_file = "PREDICTED_DOWNREGULATED_ProjE_breast.csv"

# Export the DataFrame to CSV
df.to_csv(csv_file, index=False)

print(f"Data has been exported to {csv_file}.")

Data has been exported to PREDICTED_DOWNREGULATED_ProjE_breast.csv.
